# Dates

In any language, these dates always seem to get tricky. Let's play with dates a little bit

In [62]:
import numpy as np
import pandas as pd
from datetime import date, datetime

## creating dates

In [28]:
my_year = 2025
my_month = 6
my_day = 7
my_hour = 2
my_minute = 30
my_second = 15

In [29]:
my_date = datetime(my_year, my_month, my_day, my_hour, my_minute, my_second)
my_date

datetime.datetime(2025, 6, 7, 2, 30, 15)

In [24]:
now = datetime.now()
print(f'todays date is : {now.strftime('%m/%d/%Y')}')


todays date is : 06/07/2025


In my world, I would like to just pass a date into some function. 

That function then converts it to a date and does some magic. 

In javascript, it might look something like this:

```js
const formatDate = (input) => {
    const date = new Date(input);
    const year = date.getFullYear();
    const month = String(date.getMonth() + 1).padStart(2, '0');
    const day = String(date.getDate()).padStart(2, '0');
    return `${year}/${month}/${day}`;
}
```


### Let's see why we can't use this function in python

Let's try to execute this line of code

```js
const date = new Date(input);
```

I would think that you could just do this:

```js
date_from_string = datetime('6/1/2025')
```

If you try that, you will get this error:

<div style="width: 100%; text-align: center;"><span style="color: red">'str'</span> object cannot be interpreted as an integer 💥</div>

Here is what we need to do instead

We need to use the <span style="font-size:1.1rem; color: steelblue;">strptime</span> function

My best guess is that it is 'string parsed from time'

It takes two arguments, your string, and the format. We'll talk about formats more as well

## Format String

- The format string uses special codes (e.g., %Y, %m, %d, %H, %M, %S) to indicate the parts of the date and time.
- For example:
  - %Y: Year with century (e.g., 2023)
  - %y: Year without century (e.g., 23)
  - %m: Month as a zero-padded decimal number (e.g., 01 for January)
  - %d: Day of the month as a zero-padded decimal number (e.,g 05) 
  - %H: Hour (24-hour clock)
  - %I: Hour (12-hour clock)
  - %M: Minute
  - %S: Second
  - %p: Locale's equivalent of AM or PM
  - %a: Weekday as locale's abbreviated name
  - %A: Weekday as locale's full name
  - %b: Month as locale's abbreviated name
  - %B: Month as locale's full name

## Solution

In [41]:
date_from_string = datetime.strptime('6/1/2025', '%m/%d/%Y')
date_from_string

datetime.datetime(2025, 6, 1, 0, 0)

If you try to use this function with only the date, you will get this error:

![strptime_error](../images/strptime_error.png)

Let's check out the results

In [45]:
print(date_from_string)
print(type(date_from_string))
    

2025-06-01 00:00:00
<class 'datetime.datetime'>


In [47]:
print(f"the month is: {date_from_string.month}")
print(f"the day is: {date_from_string.day}")

the month is: 6
the day is: 1


To make our function work, we need to check if the input to the function is a string or a date and convert it properly

In [48]:
def format_date(date):
    if isinstance(date, str):
        date = datetime.strptime(date, '%m/%d/%Y') # we are going to assume the format is coming from some basic json '6/2/2025'
    # now we know we are working with a datetime object
    month = date.month
    day = date.day
    year = date.year
    return f"{month:02d}/{day:02d}/{year}"

In [49]:
print(format_date('6/1/2025'))

06/01/2025


## Last Year Today

In the Grocery world, if we are comparing to last year, we need to compare the same day of the week last year, not just this calendar date last year.

Let's take for example that today is 6/7/2025. It's a Saturday. 

If I were comparing my sales to last year, last years date would actually be a Friday. 

![today](../images/calendar_today.png) ![today](../images/calendar_lastyear.png)

If I wanted to compare against the same Saturday of last year, the date would actually be the 8th

How can we generate that in Python🙄

In [53]:
this_years_sales_date = datetime.strptime('6/7/2025', '%m/%d/%Y') # remember the year is capitalized

In [56]:
this_years_week = this_years_sales_date.isocalendar().week
print(f"This year's week is: {this_years_week}")

This year's week is: 23


In [59]:
# assuming the week start is Monday, so Saturday is 6
# this is the python default
# fortunately for us the week start financially does not matter for this case
this_years_day_of_week = this_years_sales_date.isocalendar().weekday
print(f"The day of the week is: {this_years_day_of_week}")

The day of the week is: 6


In [61]:
last_years_year = this_years_sales_date.year - 1

⁉️ Now we have the year, the week number, and the day of the week that correspondes to last year, this should be pretty easy, right?

We can use the <span style="color:steelblue">datetime.date.fromisocalendar()</span> function

In [63]:
last_years_sales_date = date.fromisocalendar(last_years_year, this_years_week, this_years_day_of_week)
print(f"Last year's sales date is: {last_years_sales_date}")

Last year's sales date is: 2024-06-08


🎂 Perfect!!!

### Small Problem

#### Why Week Numbers can differ year to year

- The ISO calendar (which is what isocalendar() and fromisocalendar() use) is based on weeks starting on Monday and week 1 is the week containing the year’s first Thursday.

- Years can have 52 or 53 weeks, depending on how the calendar lines up.

- Dates near the start or end of the year can have the same calendar date but a different ISO week number in different years.

##### Example

- January 1, 2024 is in ISO week 1.

- January 1, 2023 is in ISO week 52 of 2022!

- The week that is “week 23” in 2025 might cover different date ranges than “week 23” in 2024, depending on leap years and where the first week starts.

We need to find the correct corresponding week to last year to make this work correctly

In [66]:
last_year = this_years_sales_date.replace(year=this_years_sales_date.year - 1)
last_years_week = last_year.isocalendar().week
print(f"Last year's week is: {last_years_week}")

Last year's week is: 23


## Solution

Now we should have a real solution that takes into consideration leap years

In [67]:
last_years_sales_date = datetime.fromisocalendar(last_year.year, last_years_week, this_years_day_of_week)
print(f"Last year's sales date is: {last_years_sales_date}")

Last year's sales date is: 2024-06-08 00:00:00


After all that, I just need a function that gives me the date for last year today

In [69]:
def last_years_sales_date(date):
    if isinstance(date, str):
        date = datetime.strptime(date, '%m/%d/%Y')
    this_years_day_of_week = date.isocalendar().weekday
    last_year = date.replace(year=date.year - 1)
    last_years_week = last_year.isocalendar().week
    return datetime.fromisocalendar(last_year.year, last_years_week, this_years_day_of_week)

In [71]:
lysd = last_years_sales_date('6/7/2025')
print(f"Last year's sales date is: {lysd}")

Last year's sales date is: 2024-06-08 00:00:00


## manipulation

In [25]:
today = now

In [30]:
month = today.strftime('%m')
print(f'the month is: {month}')
print(f'the single digit month is: {int(month)}')

the month is: 06
the single digit month is: 6


In [33]:
# this could also be written as:
print(f'the month is: {datetime.now().strftime('%m')}')
print(f'the single digit month is: {int(datetime.now().strftime('%m'))}')

the month is: 06
the single digit month is: 6


In [36]:
print(f'the day of the month is: {today.strftime('%d')}')
print(f'the single digit day of the month is: {int(today.strftime('%d'))}')

the day of the month is: 07
the single digit day of the month is: 7


In [74]:
yesterday = today - pd.Timedelta(days=1)
print(f"Yesterday's date is: {yesterday.strftime('%m/%d/%Y')}")

string_version_of_yesterday = yesterday.strftime('%m/%d/%Y')
print(f"String version of yesterday's date is: {string_version_of_yesterday}")

Yesterday's date is: 06/06/2025
String version of yesterday's date is: 06/06/2025


## series

In [4]:
my_ser = pd.Series(['Nov 3, 1990', '2000-01-01', None])

In [5]:
my_ser

0    Nov 3, 1990
1     2000-01-01
2           None
dtype: object

In [9]:
pd.to_datetime(my_ser, format='mixed')

0   1990-11-03
1   2000-01-01
2          NaT
dtype: datetime64[ns]

In [10]:
my_ser

0    Nov 3, 1990
1     2000-01-01
2           None
dtype: object